# Transfer Learning

Closely modeled after fast.ai's [NLP course](https://github.com/fastai/course-nlp).

### Settings & Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os.path
from fastai.text import *

### Data Preprocessing

In [3]:
path = "./Replications/Davidson2017/data"

In [8]:
bs = 128  # Batch size

# If files have been saved, load them.
if os.path.exists(path + "/data_lm_export.pkl"):
    
    print("Loading saved data files...\n")
    
    data_lm = load_data(path, 'data_lm_export.pkl')
    data_clas = load_data(path, 'data_clas_export.pkl', bs=bs)
    
else:
    
    print("Files not found. Creating data files...\n")
    
    # Language model data
    data_lm = TextLMDataBunch.from_csv(path, 'labeled_data.csv', text_cols='tweet')
    # Classifier model data
    data_clas = TextClasDataBunch.from_csv(path, 'labeled_data.csv', text_cols='tweet', label_cols='class', 
                                           vocab = data_lm.train_ds.vocab, bs=bs)
    
    # Save files
    data_lm.save('data_lm_export.pkl')
    data_clas.save('data_clas_export.pkl')

Loading saved data files...



In [9]:
data_lm.show_batch()

idx,text
0,"security , please ignore any foreign threats and focus your sole attention on our police force # xxup xxunk # xxup xxunk xxbos xxmaj this bitch @lvl_7_eevee buys xxunk water . xxbos xxunk you a bitch tho i know for sure i 'm not . xxmaj you 'd get your ass beat by any of my bitches that 's fact 😂 😂 xxbos xxmaj that 's yo bitch xxrep 5"
1,then be xxunk when she taking care of it by herself ? xxbos i coulda xxunk i was the last bitch on tunchi ig 💁 ✌ ️ 😘 xxbos @nasshrodriguez @linndaroose always down patty never been a pussy 😼 xxbos “ xxunk _ : xxmaj that pussy so wet she got ta dry xxunk telling my business xxbos i 'm hooked on monkey xxunk . xxbos a hoe gon na
2,rick 's positions are xxunk by xxunk … xxbos xxmaj these hoes is lost xxup rt xxunk : @1bcarter hahaha how much you wanna bet she do n't get the job lmao xxbos xxmaj what ever happened to the xxmaj teen xxmaj titans of my day and why must they mock me with this xxmaj teen xxmaj titans xxmaj go ? xxbos “ @liidurk _ _ : xxmaj females get
3,"t.co / xxunk xxbos xxmaj xxunk xxmaj banks still getting paid for these xxmaj america 's next top model re runs , this bitch so bossy 👠 xxunk xxbos xxmaj leave it up to me it would 've been in the trash xxrep 4 . xxmaj all of it . xxbos xxmaj wallah i 'll drop kick this bitch xxbos xxmaj crystal got all the hoes . xxbos @coolstoryisaac hahaha"
4,than xxunk you want but aye i 'm gon na have fun bitch so xxup suck a xxup xxunk ! ! xxbos “ xxunk xxmaj and happy to be hoes too . xxmaj xxunk than being in denial xxbos @denytheprophecy that 's a dragon you retard xxbos # xxunk has a highschool graduation rate near 50 % and that 's why they have so many uninformed # teabaggers # morningjoe


In [10]:
data_clas.show_batch()

text,target
xxbos xxmaj was finna slit my eyebrows up in the shop but na xxrep 6 h . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n . \n,2
xxbos xxup rt xxunk : i xxmaj be xxmaj like \n \n 🚗 xxrep 13 _ 🚓 \n xxmaj fuck ! xxmaj here come the xxunk \n \n _ _ 🚗 xxrep 6 _ 🚓 _ _ \n xxmaj be cool son \n \n _ _ _ 🚗 🚓 xxrep 5 _ \n xxmaj man fuck \n \n _ _ _ 🚓 _,1
xxbos xxup rt xxunk : xxup rt if you listen to these bands 😊 \n \n xxup xxunk \n xxup xxunk \n xxup xxunk \n xxup atl \n xxup xxunk \n xxup xxunk \n xxup xxunk \n xxup xxunk \n xxup fob \n xxup xxunk \n xxmaj blink xxunk \n xxup xxunk \n xxup xxunk \n,2
"xxbos xxup rt xxunk : “ xxmaj daddy , xxmaj batman ’s parents are dead . ” \n \n “ xxmaj yup . ” \n \n “ xxmaj and xxmaj superman ’s . ” \n \n “ xxmaj yup . ” \n \n “ xxmaj what am i going to be when you die ? ” \n \n “ xxmaj sad ho …",2
xxbos xxup rt xxunk : xxmaj all tha bad bitches need to go to @currensy_spitta concert in xxunk tonight . ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️ ✈ ️,1


### Fine-Tuning

In [19]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5).to_fp16(clip=0.1)
learn.fit_one_cycle(1, 1e-2,  moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.858530,4.310792,0.267954,00:12


In [20]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-4,1e-3), moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,4.320427,4.196193,0.280901,00:15
1,4.187600,4.094646,0.291130,00:15
2,4.065797,4.046175,0.295710,00:16
3,3.952305,4.027632,0.298496,00:16
4,3.905034,4.026507,0.299631,00:16
